In [1]:
from pycaret.classification import *
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM, Dropout, Conv1D
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect("Downloads/stocks.db")
df2 = pd.read_sql("SELECT * FROM Table_KOSDAQ Limit 500000", con, index_col = 'index') #csv �����ּ� �Է�
df = df2.reset_index(drop=True)

In [3]:
column_table = ['open','high','low','close', 'vol', 'agg_price', 'foreigner_limit', 'kospi', 'kosdaq', 'MSCI', 'DowJones', 'Nasdaq', 'HangSeng']
dfx1 = df[['open','high','low','close', 'vol', 'agg_price', 'foreigner_limit', 'kospi', 'kosdaq', 'MSCI', 'DowJones', 'Nasdaq', 'HangSeng']].round(2)
dfx2 = df[['open','high','low','close', 'vol', 'agg_price', 'foreigner_limit', 'kospi', 'kosdaq', 'MSCI', 'DowJones', 'Nasdaq', 'HangSeng']].round(2)

In [4]:

label = [0]*(len(dfx2['open'])-1)

for col in column_table:
    for i in range(len(dfx2['open'])-1):
        term = int(dfx1[col].iloc[i+1])-int(dfx1[col].iloc[i])#���ð�-������
        dfx2.at[i+1,col] = term
               
        if dfx2['close'].iloc[i+1] >=0:
            label[i-1] = 1 #mean +
    
        elif dfx2['close'].iloc[i+1] <0:
            label[i-1] = 0 #mean-

dfx2 = dfx2.drop([dfx2.index[0]])   
dfx2['label'] = label
print(dfx2.head(20))
print("1OK")
#print("(stationary, binary)")

length = len(dfx2)

def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data,0)-np.min(data,0)
    return numerator/(denominator+1e-7)

exdf = dfx2.drop(['label'], axis=1)

#scail
print("2OK")
dfx = exdf
#dfx = MinMaxScaler(exdf)
dfy = dfx2[['label']] #binary
x = dfx.values.tolist()
y = dfy.values.tolist()

x_train = []
y_train = []
x_test = []
y_test = []

window_size = 20 #size..

train_size = int(len(y)*0.7)
xtrain = x[0:train_size]
ytrain = y[0:train_size]
xtest = x[train_size:len(x)]
ytest = y[train_size:len(y)]
print("3OK")

    open  high  low  close      vol  agg_price  foreigner_limit  kospi  \
1    -60    35    5     65    17344    1300000              1.0   24.0   
2     85    40   55     15    18434     300000              0.0    1.0   
3    -10   280  -15    120  2209722    2400000              0.0  -11.0   
4    140  -165  120    -15 -1876670    -300000              0.0    0.0   
5    -35   -30 -150   -125   548683   -2500000              0.0   -3.0   
6   -125  -160  -55    -75  -663689   -1500000              0.0   -8.0   
7    -85   -25   10     30  -250105     600000              0.0    7.0   
8     15   -25  -70    -85   168117   -1700000              0.0  -23.0   
9    -50   -75  -55    -65  -162878   -1300000             -1.0   -2.0   
10   -70   -20   40     45  -193980     900000              0.0   -1.0   
11    45   160   35    115  1118730    2300000              0.0   12.0   
12   110   -55   30    -35 -1074434    -700000              1.0  -22.0   
13   -50   155   30    110   565435   

In [5]:
"""
#train
for i in range(len(ytrain)-window_size):
    _x1 = xtrain[i:i+window_size]
    _y1 = ytrain[i+window_size]
    x_train.append(_x1)
    y_train.append(_y1)
    
#test
for j in range(len(ytest)-window_size):
    _x2 = xtest[j:j+window_size]
    _y2 = ytest[j+window_size]
    x_test.append(_x2)
    y_test.append(_y2)
"""
    
x_train=np.array(xtrain)
y_train=np.array(ytrain)
x_test=np.array(xtest)
y_test=np.array(ytest)

#model = load_model('./stocks/CheckPoint/model_1_1.h5')

In [9]:
column_table = ['open','high','low','close', 'vol', 'agg_price', 'foreigner_limit', 'kospi', 'kosdaq', 'MSCI', 'DowJones', 'Nasdaq', 'HangSeng', 'label']

x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)

ml_data = pd.concat([x_train,y_train],axis=1)
ml_data.columns = column_table

#ml_data = ml_data.drop(['close'], axis=1)

print(ml_data.head())

    open   high    low  close        vol  agg_price  foreigner_limit  kospi  \
0  -60.0   35.0    5.0   65.0    17344.0  1300000.0              1.0   24.0   
1   85.0   40.0   55.0   15.0    18434.0   300000.0              0.0    1.0   
2  -10.0  280.0  -15.0  120.0  2209722.0  2400000.0              0.0  -11.0   
3  140.0 -165.0  120.0  -15.0 -1876670.0  -300000.0              0.0    0.0   
4  -35.0  -30.0 -150.0 -125.0   548683.0 -2500000.0              0.0   -3.0   

   kosdaq  MSCI  DowJones  Nasdaq  HangSeng  label  
0     2.0   8.0       0.0    -9.0    -136.0      1  
1     5.0  -1.0       0.0     8.0     -38.0      1  
2     2.0  -6.0       0.0    15.0      90.0      0  
3     3.0  -1.0       0.0    -6.0     -37.0      0  
4     1.0   0.0       0.0   -33.0      31.0      0  


In [10]:
clf = setup(ml_data, target='label')

,Description,Value
0,session_id,7844
1,Target,label
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(349999, 14)"
5,Missing Values,False
6,Numeric Features,13
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [21]:
#model compare
best3 = compare_models(sort='AUC', n_select=2)
best3

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.6204,0.6728,0.7034,0.6213,0.6598,0.2341,0.2364,17.0360
lightgbm,Light Gradient Boosting Machine,0.6086,0.6600,0.7271,0.6049,0.6604,0.2076,0.2125,0.3140
et,Extra Trees Classifier,0.6119,0.6599,0.6442,0.6254,0.6347,0.2209,0.2210,4.1940
rf,Random Forest Classifier,0.6087,0.6574,0.6422,0.6223,0.6321,0.2144,0.2146,6.0910
gbc,Gradient Boosting Classifier,0.5666,0.5962,0.7536,0.5644,0.6454,0.1169,0.1252,6.5090
ada,Ada Boost Classifier,0.5456,0.5656,0.7163,0.5506,0.6226,0.0755,0.0798,1.5050
dt,Decision Tree Classifier,0.5530,0.5521,0.5724,0.5730,0.5727,0.1041,0.1041,0.4210
qda,Quadratic Discriminant Analysis,0.5276,0.5360,0.9291,0.5277,0.6730,0.0164,0.0294,0.3770
lda,Linear Discriminant Analysis,0.5283,0.5303,0.8830,0.5296,0.6621,0.0226,0.0328,0.1570
nb,Naive Bayes,0.5253,0.5269,0.9793,0.5249,0.6835,0.0064,0.0201,0.1080


[XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.300000012, max_delta_step=0, max_depth=6,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=100, n_jobs=-1, num_parallel_tree=1,
               objective='binary:logistic', random_state=7844, reg_alpha=0,
               reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='auto',
               use_label_encoder=True, validate_parameters=1, verbosity=0),
 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
                random_state=7

In [23]:
tuned_top3 = [tune_model(i) for i in best3]
tuned_top3

#xgboost = create_model('xgboost')
#lightgbm = create_model('lightgbm')
#et = create_model('et')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6283,0.6777,0.6741,0.6369,0.6550,0.2529,0.2534
1,0.6227,0.6755,0.6798,0.6291,0.6535,0.2406,0.2415
2,0.6220,0.6714,0.6730,0.6300,0.6508,0.2397,0.2403
3,0.6219,0.6706,0.6765,0.6291,0.6519,0.2394,0.2401
4,0.6193,0.6724,0.6757,0.6264,0.6501,0.2341,0.2349
5,0.6271,0.6807,0.6828,0.6333,0.6572,0.2497,0.2505
6,0.6241,0.6755,0.6817,0.6303,0.6550,0.2435,0.2444
7,0.6265,0.6795,0.6745,0.6347,0.6540,0.2490,0.2496
8,0.6233,0.6749,0.6763,0.6306,0.6527,0.2422,0.2429
9,0.6274,0.6814,0.6858,0.6330,0.6583,0.2501,0.2511


[XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.5, max_delta_step=0, max_depth=11,
               min_child_weight=3, missing=nan, monotone_constraints='()',
               n_estimators=260, n_jobs=-1, num_parallel_tree=1,
               objective='binary:logistic', random_state=7844, reg_alpha=0.05,
               reg_lambda=0.05, scale_pos_weight=37.80000000000001,
               subsample=0.5, tree_method='auto', use_label_encoder=True,
               validate_parameters=1, verbosity=0),
 LGBMClassifier(bagging_fraction=0.5, bagging_freq=5, boosting_type='gbdt',
                class_weight=None, colsample_bytree=1.0, feature_fraction=1.0,
                importance_type='split', learning_rate=0.2, max_depth=-1,
                min_child_samples=71, min_child_weight=0.001, min_split_gain=0.7

In [24]:
blender = blend_models(estimator_list = tuned_top3,fold = 10, method = 'soft') 
blender

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5793,0.6542,0.8372,0.5663,0.6756,0.1365,0.1592
1,0.5731,0.6454,0.8125,0.5640,0.6658,0.1254,0.1423
2,0.5738,0.6459,0.8177,0.5640,0.6676,0.1264,0.1443
3,0.5811,0.6538,0.8391,0.5675,0.6771,0.1402,0.1634
4,0.5784,0.6486,0.8276,0.5666,0.6726,0.1354,0.1558
5,0.5813,0.6541,0.8316,0.5684,0.6752,0.1413,0.1628
6,0.5747,0.6495,0.8227,0.5643,0.6694,0.1279,0.1469
7,0.5798,0.6554,0.8312,0.5673,0.6743,0.1382,0.1595
8,0.5793,0.6497,0.8202,0.5679,0.6711,0.1379,0.1568
9,0.5770,0.6536,0.8276,0.5656,0.6719,0.1325,0.1528


VotingClassifier(estimators=[('xgboost',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            gpu_id=-1, importance_type='gain',
                                            interaction_constraints='',
                                            learning_rate=0.5, max_delta_step=0,
                                            max_depth=11, min_child_weight=3,
                                            missing=nan,
                                            monotone_constraints='()',
                                            n_estimators=260, n_jobs=-1,
                                            num_pa...
                                             importance_type='split',
                                            

In [14]:
evaluate_model(estimator=blender)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [16]:
plot_model(estimator=blender, plot = 'feature')

TypeError: Feature Importance and RFE plots not available for estimators that doesnt support coef_ or feature_importances_ attribute.

In [ ]:
final = finalize_model(blender)

In [ ]:
column_table = ['open','high','low','close', 'vol', 'agg_price', 'foreigner_limit', 'kospi', 'kosdaq', 'MSCI', 'DowJones', 'Nasdaq', 'HangSeng', 'label']

x_test = pd.DataFrame(x_test)
y_test = pd.DataFrame(y_test)

ml_data2 = pd.concat([x_test,y_test],axis=1)
ml_data2.columns = column_table
predict_cat = predict_model(final, ml_data2)

In [ ]:
predict_cat.head()

In [ ]:
#plot_model(estimator = final, plot = 'confusion_matrix')
# testset evaluation
from pycaret.utils import check_metric
check_metric(predict_cat['label'], predict_cat['Label'], metric = 'Accuracy')
#check_metric(predict_cat['label'], predict_cat['Label'], metric = 'Accuracy')